In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be s

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [3]:
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, random_split, DataLoader
from PIL import Image
import torchvision.models as models
from tqdm.notebook import tqdm
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torchvision.utils import make_grid

import os
import re
import requests

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [4]:
model_category = 'Men Tshirts'

In [ ]:
train_df_Men_Tshirts = pd.read_csv('../../Preprocessor-FillNA/output/train_fillna_Men_Tshirts_effnet_b5_4epochs.csv')
# train_df_Men_Tshirts = train_df_Men_Tshirts.head(100)

In [6]:
attr_columns = train_df_Men_Tshirts.filter(like='attr_').columns.to_list() # Adjust if more attributes exist
print(attr_columns)

['attr_1', 'attr_2', 'attr_3', 'attr_4', 'attr_5']


In [7]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder for each attribute column
label_encoders = {}
for column in attr_columns:
    le = LabelEncoder()
    train_df_Men_Tshirts[column] = le.fit_transform(train_df_Men_Tshirts[column])
    label_encoders[column] = le  # Store the encoder for inverse transformation later if needed

# Check the updated DataFrame
# train_df_Men_Tshirts = train_df_Men_Tshirts.head()

In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df1, val_df1 = train_test_split(train_df_Men_Tshirts, test_size=0.15, random_state=42)

In [9]:
train_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,image_path
3968,3968,Men Tshirts,5,2,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/003...
2881,2881,Men Tshirts,5,0,1,0,2,1,/kaggle/input/visual-taxonomy/train_images/002...
4216,4216,Men Tshirts,5,1,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/004...
5184,5184,Men Tshirts,5,0,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/005...
6006,6006,Men Tshirts,5,2,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/006...
...,...,...,...,...,...,...,...,...,...
3772,3772,Men Tshirts,5,2,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/003...
5191,5191,Men Tshirts,5,3,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/005...
5226,5226,Men Tshirts,5,3,1,0,2,1,/kaggle/input/visual-taxonomy/train_images/005...
5390,5390,Men Tshirts,5,1,1,0,0,1,/kaggle/input/visual-taxonomy/train_images/005...


In [10]:
val_df1

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,image_path
1414,1414,Men Tshirts,5,1,1,0,0,1,/kaggle/input/visual-taxonomy/train_images/001...
4610,4610,Men Tshirts,5,3,1,0,2,1,/kaggle/input/visual-taxonomy/train_images/004...
2862,2862,Men Tshirts,5,1,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/002...
4078,4078,Men Tshirts,5,2,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/004...
3964,3964,Men Tshirts,5,3,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/003...
...,...,...,...,...,...,...,...,...,...
1545,1545,Men Tshirts,5,0,1,0,0,1,/kaggle/input/visual-taxonomy/train_images/001...
3483,3483,Men Tshirts,5,0,1,0,0,1,/kaggle/input/visual-taxonomy/train_images/003...
1858,1858,Men Tshirts,5,1,0,1,1,1,/kaggle/input/visual-taxonomy/train_images/001...
6611,6614,Men Tshirts,5,1,1,1,1,0,/kaggle/input/visual-taxonomy/train_images/006...


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np

# Define custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, is_test=False):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test  # Flag to indicate if it's test set without labels

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.img_dir + self.dataframe.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        if self.is_test:  # For test set, just return the image without labels
            return image
        
        # For train/validation set, return image and labels
        labels = self.dataframe.iloc[idx][attr_columns].values
        labels = labels.astype(np.int64)  # Ensure labels are integers
        labels = torch.tensor(labels, dtype=torch.long)
        
        return image, labels
    
    
# Transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Apply Color Jitter
    transforms.RandomHorizontalFlip(),  # Apply Horizontal Flip with 50% probability
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
    
train_dataset = CustomDataset(dataframe=train_df1, img_dir='', transform=transform)
val_dataset = CustomDataset(dataframe=val_df1, img_dir='', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [12]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [13]:
from torchvision import models
available_models = dir(models)

# Filter out private members and display model names
available_models = [model for model in available_models if not model.startswith('_')]
print("Available models in torchvision.models:")
print(available_models)

Available models in torchvision.models:
['AlexNet', 'AlexNet_Weights', 'ConvNeXt', 'ConvNeXt_Base_Weights', 'ConvNeXt_Large_Weights', 'ConvNeXt_Small_Weights', 'ConvNeXt_Tiny_Weights', 'DenseNet', 'DenseNet121_Weights', 'DenseNet161_Weights', 'DenseNet169_Weights', 'DenseNet201_Weights', 'EfficientNet', 'EfficientNet_B0_Weights', 'EfficientNet_B1_Weights', 'EfficientNet_B2_Weights', 'EfficientNet_B3_Weights', 'EfficientNet_B4_Weights', 'EfficientNet_B5_Weights', 'EfficientNet_B6_Weights', 'EfficientNet_B7_Weights', 'EfficientNet_V2_L_Weights', 'EfficientNet_V2_M_Weights', 'EfficientNet_V2_S_Weights', 'GoogLeNet', 'GoogLeNetOutputs', 'GoogLeNet_Weights', 'Inception3', 'InceptionOutputs', 'Inception_V3_Weights', 'MNASNet', 'MNASNet0_5_Weights', 'MNASNet0_75_Weights', 'MNASNet1_0_Weights', 'MNASNet1_3_Weights', 'MaxVit', 'MaxVit_T_Weights', 'MobileNetV2', 'MobileNetV3', 'MobileNet_V2_Weights', 'MobileNet_V3_Large_Weights', 'MobileNet_V3_Small_Weights', 'RegNet', 'RegNet_X_16GF_Weights', '

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

class MultiOutputModelVit(nn.Module):
    def __init__(self, num_classes):
        super(MultiOutputModelVit, self).__init__()
        
        # Load a pretrained ViT model
        self.base_model = models.vit_l_16(pretrained=True)
        
        # Remove the original classification head
        self.base_model.heads = nn.Identity()  # Replace with identity to extract embeddings
        
        # Define output layers for each attribute dynamically
        self.output_layers = nn.ModuleDict()
        for attr, n_classes in num_classes.items():
            self.output_layers[attr] = nn.Linear(self.base_model.hidden_dim, n_classes)

    def forward(self, x):
        # Pass input through the ViT model
        x = self.base_model(x)  # Get embeddings directly
        
        # Compute output for each attribute
        outputs = {attr: layer(x) for attr, layer in self.output_layers.items()}
        
        return list(outputs.values())

# Example usage
# Define number of classes for each attribute
num_classes = {attr: train_df_Men_Tshirts[attr].nunique() for attr in attr_columns}

# Initialize the model
model = MultiOutputModelVit(num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


Downloading: "https://download.pytorch.org/models/vit_l_16-852ce7e3.pth" to /root/.cache/torch/hub/checkpoints/vit_l_16-852ce7e3.pth
100%|██████████| 1.13G/1.13G [00:13<00:00, 89.8MB/s]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Training loop
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, save_path="../../Models/Men Tshirts/effnet_Men_Tshirts_vit_l_16.pth"):
    scheduler = optim.lr_scheduler.StepLR(optimizer = optimizer , step_size=5, gamma=0.5)
    best_val_loss = float('inf')
    for epoch in tqdm(range(num_epochs)):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
            optimizer.zero_grad()
            outputs = model(images)
            loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Move tensors to the correct device
                outputs = model(images)
                loss = sum([criterion(output, label) for output, label in zip(outputs, labels.T)])
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        print(f'Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')
        
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved with Val Loss: {best_val_loss:.4f} at Epoch {epoch+1}")

# Run training
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=6)

model.load_state_dict(torch.load('../../Models/Men Tshirts/effnet_Men_Tshirts_vit_l_16.pth'))

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1, Train Loss: 1.2975132243003251
Epoch 1, Val Loss: 1.141742398057665
Model saved with Val Loss: 1.1417 at Epoch 1
Epoch 2, Train Loss: 0.8441762173731711
Epoch 2, Val Loss: 0.8918081292084286
Model saved with Val Loss: 0.8918 at Epoch 2
Epoch 3, Train Loss: 0.6595700372377208
Epoch 3, Val Loss: 0.9246225578444345
Epoch 4, Train Loss: 0.5627330123914955
Epoch 4, Val Loss: 0.9168126583099365
Epoch 5, Train Loss: 0.498313577295585
Epoch 5, Val Loss: 0.9547317445278167
Epoch 6, Train Loss: 0.3506698274087412
Epoch 6, Val Loss: 0.9685685110943658


<All keys matched successfully>